# How to start

Before starting you must:
- Ensure that `scipp` and `mantid` are on your `PYTHONPATH`.
- Generate the `config.py` file using `make_config.py`. Refer to the `README.md` or `python make_config.py --help` for information.
- Install dependencies : `conda install fabio tifffile` (used for image handling)

For `scipp` and `mantid` follow instructions at: https://scipp.readthedocs.io/en/latest/getting-started/installation.html.

Converted to use scipp and notebook from [this file](https://git.esss.dk/testbeamline/gp2/blob/1c69213b1124982bbbe762da9c6c6457a49f2a92/reduce.py) by Dimitar Tasev on 2020-01-13

In [ ]:
try:
    import scipp
    print("scipp found")
except ImportError as e:
    print("scipp is not available in the PYTHONPATH")
    raise e
    
try:
    import mantid
    print("mantid found")
except ImportError as e:
    print("mantid is not available in the PYTHONPATH")
    raise e
    
try:
    import scippconfig
    print("scippconfig found")
except ImportError as e:
    print("scippconfig is not available. Make sure you have generated it with `make_config.py`.")
    raise e

In [ ]:
# Lets get everything set up

import csv
import glob
import fabio
import os

import scipp as sc
import numpy as np
from scipp import Dim
from mantid.simpleapi import SaveNexusESS

DATA_DIR_NAME = "data_GP2"
experiment_dir = scippconfig.script_root
data_dir = os.path.join(scippconfig.script_root, DATA_DIR_NAME)

if not os.path.exists(data_dir):
    raise FileNotFoundError("The following data directory does not exist,"
                            f" check your make_config.py:\n{data_dir}")

In [ ]:
# Customisable Options:

# Whether or not to do the plotting.
do_plots = True

# defining grouping of 2D detector pixels
grouping_number = 3
nx_target = grouping_number
ny_target = grouping_number

# Rebin regions for each of the 5 frames
# in the format of [bin-start, bin-end, bin width].
# used to crop each image, before stitching them together
frame_parameters = [(15167, 23563, 64),
                    (24393, 32758, 64),
                    (33365, 40708, 64),
                    (41410, 48019, 64),
                    (49041, 55311, 64),
                    (56077, 59872, 64)]

# Used to shift the cropped frames so that their bins overlap 
# before summing them together into a single frame
frame_shift_increments = [-6630.0, -2420.0, -2253.0, -2095.0, -1946.0, -1810.0]
frame_shifts = [sum(frame_shift_increments[:i + 1]) for i in range(len(frame_shift_increments))]

# Used to rebin the summed frame in order to
# cut off frames that contain no data
rebin_parameters = (8500, 43000, 64)

In [ ]:
# Some helper functions
def read_x_values(tof_file):
    """
    Reads the TOF values from the CSV into a list
    """
    tof_values = []
    with open(tof_file) as fh:
        csv_reader = csv.reader(fh, delimiter='\t')
        next(csv_reader, None)  # skip header
        for row in csv_reader:
            tof_values.append(float(row[1]))
    return tof_values


def _load_tiffs(tiff_dir):
    if not os.path.isdir(tiff_dir):
        raise RuntimeError(tiff_dir + " is not directory")
    stack = []
    path_length = len(tiff_dir) + 1
    filenames = sorted(glob.glob(tiff_dir + "/*.tiff"))
    nfiles = len(filenames)
    count = 0
    print(f"Loading {nfiles} files from '{tiff_dir}'")
    for filename in filenames:
        count += 1
        print('\r{0}: Image {1}, of {2}'.format(filename[path_length:], count, nfiles), end="")
        img = fabio.open(os.path.join(tiff_dir, filename))
        stack.append(np.flipud(img.data))

    return np.array(stack)

def tiffs_to_variable(tiff_dir):
    """
    Loads all tiff images from the directory into a scipp Variable.
    """
    stack = _load_tiffs(tiff_dir)
    data = stack.astype(np.float64).reshape(stack.shape[0], stack.shape[1]*stack.shape[2])
    return sc.Variable([Dim.Tof, Dim.Spectrum], 
                       values=data, variances=data)

def stitch(data_array, frame_parameters, rebin_parameters):
    """
    Stitches the 5 different frames data.
    
    It crops out each frame, then shifts it so that all frames align,
    and then rebins to the common bins used for all frames.
    """
    frames = []

    rebin_params = sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))
    
    for i, (slice_bins, shift_parameter) in enumerate(zip(frame_parameters, frame_shifts)):
        bins = sc.Variable([Dim.Tof], values=np.arange(*slice_bins, dtype=np.float64))
        # Rebins the whole data to crop it to frame bins
        rebinned = sc.rebin(data_array, Dim.Tof, bins)
        # Shift the frame backwards to make all frames overlap
        rebinned.coords[Dim.Tof] += shift_parameter
        # Rebin to overarching coordinates so that the frame coordinates align
        rebinned = sc.rebin(rebinned, Dim.Tof, rebin_params)

        frames.append(rebinned)

    for f in frames[1:]:
        frames[0] += f

    return frames[0]

def make_detector_groups(nx_original, ny_original, nx_target, ny_target):
    element_width_x = nx_original // nx_target
    element_width_y = ny_original // ny_target
    
    # To contain our new spectra mappings
    grid = np.zeros((nx_original, ny_original), dtype=np.float64)

    for i in range(0, nx_target):
        x_start = i * element_width_x
        x_end = (i + 1) * element_width_x
        
        for j in range(0, ny_target):
            y_start = j * element_width_y
            y_end = (j + 1) * element_width_y

            vals = np.full((element_width_x, element_width_y), i + j * nx_target, dtype=np.float64)
            grid[x_start:x_end, y_start:y_end] = vals
    
    return sc.Variable([Dim.Spectrum], values=grid.ravel())

In [ ]:
# let's get the process started:
tofs_path = os.path.join(data_dir, 'metadata', 'GP2_BCC_time_values.txt')
sample_path = os.path.join(data_dir, 'Timeslices WFM BBC Steel')
ob_path = os.path.join(data_dir, 'Timeslices WFM Open Beam')
instrument_file = os.path.join(experiment_dir, 'IDF', 'V20_Definition_GP2.xml')

ds = sc.Dataset()

# Load X values from the TOF file
ds.coords[Dim.Tof] = sc.Variable([Dim.Tof], unit=sc.units.us, values=read_x_values(tofs_path))
ds.coords[Dim.Tof] *= 1e3

In [6]:
# Load the data into the dataset
ds["sample"] = tiffs_to_variable(sample_path)
ds["reference"] = tiffs_to_variable(ob_path)

sc.plot.plot(sc.sum(ds, Dim.Spectrum)) if do_plots else None

GP2_007-004-WFM-open-beam_2000.tiff: Image 2001, of 2001Loading 2001 files from '/home/david/Scipp/gp2_data/data_GP2/Timeslices WFM Open Beam'
GP2_007-003-WFM-open-beam_2000.tiff: Image 2001, of 2001

<IPython.core.display.Javascript object>

In [7]:
# Adds a coordinate for the spectra
ds.coords[Dim.Spectrum] = sc.Variable([Dim.Spectrum], values=np.arange(ds["sample"].shape[1]))
ds

<scipp.Dataset>
Dimensions: {{spectrum, 104976}, {tof, 2001}}
Coordinates:
    tof                       float64    [μs]            (tof [bin-edges])  [15.000000, 44.990000, ..., 59985.010000, 60015.000000]
    spectrum                  int64      [dimensionless]  (spectrum)  [0, 1, ..., 104974, 104975]
Data:
    reference                 float64    [dimensionless]  (tof, spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]  [0.000000, 0.000000, ..., 0.000000, 0.000000]
    sample                    float64    [dimensionless]  (tof, spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]  [0.000000, 0.000000, ..., 0.000000, 0.000000]

In [8]:
stitched = sc.Dataset(coords={Dim.Tof: sc.Variable([Dim.Tof], values=np.arange(*rebin_parameters, dtype=np.float64))})

stitched["sample"] = stitch(ds["sample"], frame_parameters, rebin_parameters)
stitched["reference"] = stitch(ds["reference"], frame_parameters, rebin_parameters)

sc.plot.plot(sc.sum(stitched, Dim.Spectrum)) if do_plots else None

<IPython.core.display.Javascript object>

In [9]:
stitched.coords["detector_mapping"] = make_detector_groups(324, 324, nx_target, ny_target)
stitched.coords["detector_mapping"]

<scipp.VariableView>      float64    [dimensionless]  (spectrum)  [0.000000, 0.000000, ..., 8.000000, 8.000000]

In [10]:
stitched["normalized"] = stitched["sample"] / stitched["reference"]
replacement=sc.Variable(value=0.0, variance=0.0)
kwargs = {"nan" : replacement, "posinf" : replacement, "neginf" : replacement}
sc.nan_to_num(stitched["normalized"].data, out=stitched["normalized"].data, **kwargs)

<scipp.VariableView>      float64    [dimensionless]  (tof, spectrum)  [0.000000, 0.000000, ..., 0.000000, 0.000000]  [0.000000, 0.000000, ..., 0.000000, 0.000000]

In [11]:
dm1d = sc.groupby(stitched["normalized"], "detector_mapping")
grouped = sc.Dataset()
grouped["normalized_grpd"] = dm1d.sum(Dim.Spectrum)

In [12]:
# Plot the first group
grouped
sc.plot.plot(grouped["normalized_grpd"]['detector_mapping', 0]) if do_plots else None

<IPython.core.display.Javascript object>

In [13]:
# Adds the component info needed for converting units
sc.compat.mantid.load_component_info(stitched, instrument_file)

grouped.coords["source_position"] = stitched.coords["source_position"]
grouped.coords["sample_position"] = stitched.coords["sample_position"]

In [14]:
# Note: this cell is currently a workaround until 
# https://github.com/scipp/scipp/issues/823 is done.
# Then it should be possible to group the positions label directly.

# makes the position a DataConstProxy otherwise groupby won't take it
stitched["position"] = stitched.coords["position"]
dm1d = sc.groupby(stitched["position"], "detector_mapping")
position = dm1d.mean(Dim.Spectrum)

# can't do stitched.labels["position"] = position because Labels won't take a DataArray
# also can't do stitched.labels["position"] = sc.Variable(position) because then sc.convert 
# thinks stitched.labels["position"] is dimensionless (as it's actually still a DataArray)

pos = np.zeros((position.shape[0], 3))
for i, val in enumerate(position.values):
    pos[i, :] = val

# finally add it back!
grouped.coords["position"] = sc.Variable(position.dims, pos, unit=sc.units.m, dtype=position.dtype)


In [15]:
grouped["normalized_wl"] = sc.neutron.convert(grouped["normalized_grpd"], Dim.Tof, Dim.Wavelength)
grouped

<scipp.Dataset>
Dimensions: {{tof, 539}, {detector_mapping, 9}, {wavelength, 539}}
Coordinates:
    source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
    tof                       float64    [dimensionless]  (tof [bin-edges])  [8500.000000, 8564.000000, ..., 42932.000000, 42996.000000]
    sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]
    wavelength                float64    [Å μs^-1]     (detector_mapping, wavelength [bin-edges])  [1.765094, 1.778385, ..., 8.915180, 8.928471]
    detector_mapping          float64    [dimensionless]  (detector_mapping)  [0.000000, 1.000000, ..., 7.000000, 8.000000]
    position                  vector_3_float64  [m]              (detector_mapping)  [(-0.007560, -0.007560, 0.600500), (-0.007560, -0.000000, 0.600500), ..., (0.007560, -0.000000, 0.600500), (0.007560, 0.007560, 0.600500)]
Data:
    normalized_grpd           float64    [dimensionless]  (tof, detector_mapping)  [65.458559, 71.634779, ..., 0.000000, 0.000000]  [179.037722, 209.919311, ..., 0.000000, 0.000000]
    normalized_wl             float64    [dimensionless]  (wavelength, detector_mapping)  [65.458559, 71.634779, ..., 0.000000, 0.000000]  [179.037722, 209.919311, ..., 0.000000, 0.000000]
    Attributes:
            source_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, -18.450000)]
            position                  vector_3_float64  [m]              (detector_mapping)  [(-0.007560, -0.007560, 0.600500), (-0.007560, -0.000000, 0.600500), ..., (0.007560, -0.000000, 0.600500), (0.007560, 0.007560, 0.600500)]
            sample_position           vector_3_float64  [m]              ()  [(0.000000, 0.000000, 0.318500)]

In [16]:
sc.plot.plot(grouped["normalized_wl"]["detector_mapping", 0]) if do_plots else None

<IPython.core.display.Javascript object>

In [17]:
x_coords = grouped["normalized_wl"].coords["wavelength"]
x_dim = grouped["normalized_wl"].dims[0]
x = x_coords.values

# Mantid expects the data in a different shape
# which is spectrum as outer-most dimension.
y = np.transpose(grouped["normalized_wl"].values)
e = np.transpose(grouped["normalized_wl"].variances)

ws = sc.compat.mantid.to_workspace_2d(x, y, e, str(x_dim), instrument_file)
ws

Workspace2D
Title: 
Histograms: 9
Bins: 539
Histogram
X axis: Wavelength / Angstrom
Y axis: 
Distribution: False
Instrument: V20 (1900-Jan-31 to 2099-Dec-31)Instrument from: /home/david/Scipp/gp2_data/IDF/V20_Definition_GP2.xml

Run start: not available
Run end:  not available

In [18]:
# If you want to save the workspace uncomment this line
SaveNexusESS(ws, "scipp_normalized_wl.nxs")

In [19]:
fit_ds = sc.compat.mantid.fit(ws, 
                    function='name=LinearBackground,A0=5000,A1=0;name=UserFunction,Formula=h*erfc(a*(x-x0)),h=5000,a=-0.5,x0=4',
                    workspace_index=2, start_x=3.6, end_x=4.4)
fit_ds

<scipp.Dataset>
Dimensions: {}
Attributes:
    cost_function             string     [dimensionless]  ()  ["Least squares"]
    function                  string     [dimensionless]  ()  ["name=LinearBackground,A0=1415.4,A1=323.532;name=UserFunction,Formula=h*erfc(a*(x-x0)),h=1702.3,a=-16.2225,x0=3.97279"]
    status                    string     [dimensionless]  ()  ["success"]
    chi2_over_DoF             float64    [dimensionless]  ()  [0.003692]
Data:
    normalised_covariance_matrix  Dataset    [dimensionless]  ()  [<scipp.Dataset>
Dimensions: {{row, 5}}
Data:
    Name                      string     [dimensionless]  (row)  ["f0.A0", "f0.A1", ..., "f1.a", "f1.x0"]
    f0.A0                     float64    [dimensionless]  (row)  [100.000000, -99.977578, ..., 56.734733, 8.889263]
    f0.A1                     float64    [dimensionless]  (row)  [-99.977578, 100.000000, ..., -57.131905, -8.704458]
    f1.a                      float64    [dimensionless]  (row)  [56.734733, -57.131905, ..., 100.000000, 52.940305]
    f1.h                      float64    [dimensionless]  (row)  [90.458831, -90.803485, ..., 63.801922, 22.774353]
    f1.x0                     float64    [dimensionless]  (row)  [8.889263, -8.704458, ..., 52.940305, 100.000000]

]
    parameters                Dataset    [dimensionless]  ()  [<scipp.Dataset>
Dimensions: {{row, 6}}
Data:
    Error                     float64    [dimensionless]  (row)  [5526.754014, 1486.340400, ..., 0.020300, 0.000000]
    Name                      string     [dimensionless]  (row)  ["f0.A0", "f0.A1", ..., "f1.x0", "Cost function value"]
    Value                     float64    [dimensionless]  (row)  [1415.398610, 323.531656, ..., 3.972785, 0.003692]

]
    workspace                 DataArray  [dimensionless]  ()  [<scipp.DataArray>
Dimensions: {{empty, 3}, {wavelength, 60}}
Coordinates:
    detector_info             Dataset    [dimensionless]  ()  [<scipp.Dataset>
Dimensions: {{detector, 0}}
Coordinates:
    detector                  int32      [dimensionless]  (detector)  []
    spectrum                  int32      [dimensionless]  (detector)  []

]
    wavelength                float64    [Å]            (wavelength [bin-edges])  [3.612421, 3.625712, ..., 4.396539, 4.409829]
    empty                     string     [dimensionless]  (empty)  ["Data", "Calc", "Diff"]
Attributes:
    run                       PyObject   [dimensionless]  ()  [<PyObject>]
    sample                    PyObject   [dimensionless]  ()  [<PyObject>]
    instrument-name           string     [dimensionless]  ()  [""]
Data:
                              float64    [dimensionless]  (empty, wavelength)  [2583.402538, 2565.786804, ..., -71.336888, -154.059607]  [153205.758889, 156326.800027, ..., 0.000000, 0.000000]
    Attributes:
            run                       PyObject   [dimensionless]  ()  [<PyObject>]
            sample                    PyObject   [dimensionless]  ()  [<PyObject>]
            instrument-name           string     [dimensionless]  ()  [""]

]

In [20]:
sc.plot.plot(fit_ds["workspace"].value, collapse=Dim.Wavelength)

<IPython.core.display.Javascript object>